<a href="https://colab.research.google.com/github/smyyl1591/.-R2-R2-HW.ipynb/blob/main/AI_Day_HW_2_240627_Yun.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**LangChain Introduction**
**LangChain 是圍繞 LLMs 構建的框架**。 我們可以將其用於聊天機器人、Question-Answering (QA)、摘要等等。

這個函式庫的核心思想是我們可以將不同的元件 “鏈結” 在一起，以創建更多元的 LLMs 應用。 Chain 來自幾個 Module 的多個組件：

**Prompt templates**：Prompt templates 是不同類型提示的範本。例如「 chatbot 」樣式模板、ELI5 問答等

**LLMs**：像 GPT-3、Mistral、Llama、Breeze、TAIDE 等大型語言模型

**Agents**：Agents 使用 LLMs 決定應採取的動作。可以使用網路搜尋(Google Search)或計算器(Python func)之類的工具，並將所有工具包裝成一個邏輯循環的操作。

**Memory**：短期記憶、長期記憶。

In [13]:

# 安裝 langchain 和相關的 huggingface 庫
!pip install langchain transformers

import torch
from transformers import BitsAndBytesConfig

from langchain import LLMChain
from langchain.prompts import ChatPromptTemplate, PromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace
from langchain.schema import StrOutputParser



In [14]:
!pip show langchain transformers


Name: langchain
Version: 0.2.6
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 
---
Name: transformers
Version: 4.41.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: langchain-huggingface, sentence-transformers


In [1]:
from huggingface_hub import notebook_login

# chat model 需要使用 hf 的 token
notebook_login()

In [16]:
!pip install bitsandbytes
!pip install bitsandbytes==0.39.0

In [18]:
!pip install langchain transformers

import torch
from transformers import BitsAndBytesConfig
from langchain_community.llms import HuggingFacePipeline
from langchain_community.chat_models import ChatHuggingFace

MODEL_NAME = "MediaTek-Research/Breeze-7B-Instruct-v0_1"

# 量化參數
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

# llm 初始化
llm = HuggingFacePipeline.from_model_id(
    model_id=MODEL_NAME,
    task="text-generation",
    model_kwargs={
        "torch_dtype": torch.float16,
        "trust_remote_code": True,
        "device_map": "auto",
        "quantization_config": quantization_config
    },
    pipeline_kwargs={
        "max_new_tokens": 1024,
        "temperature": 0.0001,
        "top_p": 0.95,
        "do_sample": True,
        "repetition_penalty": 1.15
    }
)

# chat model 初始化
chat_llm = ChatHuggingFace(llm=llm)

ModuleNotFoundError: No module named 'langchain_community'

In [17]:
prompt = """
提問: 2022 年全球最賣座的電影是哪一部?

Let's think step by step.

解答: """
print(llm(prompt))

NameError: name 'llm' is not defined

In [18]:
prompt = """
提問: NBA 2023 年總冠軍球隊是誰?

Let's think step by step.

解答: """
print(llm(prompt))

NameError: name 'llm' is not defined

2. Prompt

一個好的 Prompt 通常包含以下四個組成部分：

指示: 告訴模型要做什麼，如何使用提供的信息，如何處理查詢，並建立輸出
範例輸入: 提供範例輸入，以向模型示範預期的內容
範例輸出: 提供對應的範例輸出
查詢: 您希望模型處理的實際輸入
以下介紹幾種在 LangChain 上使用 Pormpt 的方式

2.1 PromptTemplate


In [19]:
# 初始化提示詞模板
prompt_template = PromptTemplate.from_template("告訴我一個笑話")
message = prompt_template.format()

# 模型生成
print(llm(message))


# 初始化提示詞模板
prompt_template = PromptTemplate.from_template("告訴我關於一個{content}的{adjective}笑話。")
message = prompt_template.format(adjective="悲傷的", content="數據科學家")

# 模型生成
print(llm(message))

NameError: name 'llm' is not defined

2.2 ChatPromptTemplate

In [ ]:
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一個擁有強大能力的 AI 機器人。你的名字是{name}。"),
        ("human", "你好，你好嗎？"),
        ("ai", "我很好，謝謝！"),
        ("human", "{user_input}"),
    ]
)
messages = chat_template.format_messages(name="Bob", user_input="你叫什麼名字？")
print(chat_llm.invoke(messages).content)

2.3 混合使用

In [ ]:
system_prompt = "你是一個擁有強大能力的 AI 機器人。"
question_prompt = "告訴我關於一個{content}的{adjective}笑話。"

full_prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content=system_prompt),
    HumanMessage("你好，你好嗎？"),
    AIMessage("我很好，謝謝！"),
    HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            template=question_prompt,
            input_variables=["content", "adjective"])
    )
])
full_prompt.pretty_print()

# 3. LLM Chain

對於簡單的任務，使用單一 LLM（大型語言模型）效果很好。然而，對於更複雜的任務，通常需要鍊式多個步驟和/或模型。

在LangChain中，可以使用傳統的 LLMChain，較新且建議的方法是 LangChain 表達式語言（LCEL）。



In [ ]:
#3.1LLM chain

template = """問題: {question}

Let's think step by step.

答案: """
prompt = PromptTemplate(template=template, input_variables=["question"])


# 使用 LLM Chain 將 Prompt 與 LLM 串接起來
llm_chain = LLMChain(prompt=prompt, llm=llm)


# 將問題透過參數化的方式帶入
question = "NBA 2023 年總冠軍球隊是誰?"
print(llm_chain.invoke({"question": question})["text"])

In [ ]:
#3.2 LCEL(LangChain Expression Language)

# 需要加入 Instruction 參數，才能告訴模型你的對話結束了，因目前使用的模型上不支援 chat 模式
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一位專業的資料科學家和機器學習工程師，能夠提供專業的知識並準確地回答問題。",),
        ("human", "{question}"),
    ]
)

# 使用 LCEL 將 Prompt 與 LLM 等串接起來
llm_chain = prompt | chat_llm | StrOutputParser()

# 將問題透過參數化的方式帶入
print(llm_chain.invoke({"question": "機器學習和深度學習有什麼不同？"}))


# **作業區**
利用 Langchain 的 LLMs, Prompt 透過 LCEL 實作資訊擷取機器人。

Baseline:
從新聞描述中擷取出相關資訊
以 JSON 格式回傳，並轉為 Python Dict

Advanced（Optional）:
如何將多個 LLMChain 進行串接，將多個 LLM 透過流程化的方式結合

In [2]:
#LLM chain method

template = """問題: {question}

Let's think step by step.

答案: """

prompt = PromptTemplate(template=template, input_variables=["question"])

# 使用 LLM Chain 將 Prompt 與 LLM 串接起來
llm_chain = LLMChain(prompt=prompt, llm=llm)

# 將問題透過參數化的方式帶入
question = "請解釋 NVIDIA Omniverse Cloud Sensor RTX™是什麼?"
print(llm_chain.invoke({"question": question})["text"])






NameError: name 'PromptTemplate' is not defined

In [ ]:
# LCEL(LangChain Expression Language) method

# 需要加入 Instruction 參數，才能告訴模型你的對話結束了，因目前使用的模型上不支援 chat 模式
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一位專業的資料科學家和機器學習工程師，能夠提供專業的知識並準確地回答問題。請閱讀以下新聞並回答問題：NVIDIA 今天宣布推出 NVIDIA Omniverse Cloud Sensor RTX™，這是一組微服務，可實現物理上精確的感測器模擬，從而加速各種完全自主機器的開發。感測器是一個價值數十億美元且不斷增長的產業，為自動駕駛汽車、人形機器人、工業機械手、移動機器人和智慧空間提供理解物理世界和做出明智決策所需的數據。透過 NVIDIA Omniverse Cloud Sensor RTX，開發人員可以在實際部署之前在物理精確、真實的虛擬環境中大規模測試感測器感知和相關人工智慧（AI）軟體，從而增強安全性，同時節省時間和成本。NVIDIA Omniverse 和模擬技術副總裁 Rev Lebaredian 表示：「開發由生成式物裡 AI 驅動的安全可靠的自主機器需要在基於物理的虛擬世界中進行訓練和測試。NVIDIA Omniverse Cloud Sensor RTX 微服務將使開發人員能夠輕鬆建造工廠、城市甚至地球的大規模數位孿生，從而幫助加速下一波 AI 浪潮。」",),
        ("human", "{question}"),
    ]
)

# 使用 LCEL 將 Prompt 與 LLM 等串接起來
llm_chain = prompt | chat_llm | StrOutputParser()

# 將問題透過參數化的方式帶入
print(llm_chain.invoke({"question": "請解釋 NVIDIA Omniverse Cloud Sensor RTX™是什麼?"}))